Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [ ]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
import matplotlib.pyplot as plt

First reload the data we generated in `1_notmnist.ipynb`.

In [ ]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [ ]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

In [ ]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [ ]:
###
# Define the graph
###
batch_size = 128
fc1_nodes = 1024
lr = 0.05
lam = 0.001

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, 
                                      [batch_size, image_size * image_size])
    tf_train_labels = tf.placeholder(tf.float32, 
                                     [batch_size, num_labels])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    fc1_weight = tf.Variable(tf.truncated_normal([image_size * image_size, fc1_nodes]))
    fc1_bias = tf.Variable(tf.zeros([fc1_nodes]))
    
    fc2_weight = tf.Variable(tf.truncated_normal([fc1_nodes, num_labels]))
    fc2_bias = tf.Variable(tf.zeros([num_labels]))
    
    def model(data):
        h1 = tf.nn.relu(tf.matmul(data, fc1_weight) + fc1_bias)
        logits = tf.matmul(h1, fc2_weight) + fc2_bias
        return logits
    
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    regularizer = (tf.nn.l2_loss(fc1_weight) + tf.nn.l2_loss(fc1_bias) 
                   + tf.nn.l2_loss(fc2_weight) + tf.nn.l2_loss(fc2_bias))
    loss += lam * regularizer
    optimizer = tf.train.GradientDescentOptimizer(lr).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

###
# Run the graph
###
train_size = train_dataset.shape[0]
num_steps = train_size // batch_size * 10

with tf.Session(graph=graph) as sess:
    tf.global_variables_initializer().run()
    train_accs, valid_accs, step_hist = [], [], []
    for step in range(num_steps):
        offset = (step * batch_size) % (train_size - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, 
                     tf_train_labels: batch_labels}
        _, l, predictions = sess.run([optimizer, loss, train_prediction],
                                     feed_dict=feed_dict)
        if (step % 500 == 0):
            train_acc = accuracy(predictions, batch_labels)
            valid_acc = accuracy(valid_prediction.eval(), valid_labels)
            train_accs.append(train_acc / 100.0)
            valid_accs.append(valid_acc / 100.0)
            step_hist.append(step)
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % train_acc)
            print("Validation accuracy: %.1f%%" % valid_acc)
    
    plt.plot(step_hist, train_accs, 'b', label='training')
    plt.plot(step_hist, valid_accs, 'g', label='validation')
    plt.legend(loc="lower right")
    plt.show()
    print("Test accuracy: %.1f%%" % accuracy(
            test_prediction.eval(), test_labels))

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [ ]:
###
# Define the graph
###
batch_size = 128
fc1_nodes = 1024
lr = 0.05
lam = 0.001

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, 
                                      [batch_size, image_size * image_size])
    tf_train_labels = tf.placeholder(tf.float32, 
                                     [batch_size, num_labels])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    fc1_weight = tf.Variable(tf.truncated_normal([image_size * image_size, fc1_nodes]))
    fc1_bias = tf.Variable(tf.zeros([fc1_nodes]))
    
    fc2_weight = tf.Variable(tf.truncated_normal([fc1_nodes, num_labels]))
    fc2_bias = tf.Variable(tf.zeros([num_labels]))
    
    def model(data):
        h1 = tf.nn.relu(tf.matmul(data, fc1_weight) + fc1_bias)
        logits = tf.matmul(h1, fc2_weight) + fc2_bias
        return logits
    
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    regularizer = (tf.nn.l2_loss(fc1_weight) + tf.nn.l2_loss(fc1_bias) 
                   + tf.nn.l2_loss(fc2_weight) + tf.nn.l2_loss(fc2_bias))
    loss += lam * regularizer
    optimizer = tf.train.GradientDescentOptimizer(lr).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

###
# Run the graph
###
train_size = train_dataset.shape[0]
actual_train_size = batch_size * 3
num_steps = actual_train_size // batch_size * 100

with tf.Session(graph=graph) as sess:
    tf.global_variables_initializer().run()
    train_accs, valid_accs, step_hist = [], [], []
    for step in range(num_steps):
        offset = (step * batch_size) % (actual_train_size - batch_size + 1)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, 
                     tf_train_labels: batch_labels}
        _, l, predictions = sess.run([optimizer, loss, train_prediction],
                                     feed_dict=feed_dict)
        if (step % 50 == 0):
            train_acc = accuracy(predictions, batch_labels)
            valid_acc = accuracy(valid_prediction.eval(), valid_labels)
            train_accs.append(train_acc / 100.0)
            valid_accs.append(valid_acc / 100.0)
            step_hist.append(step)
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % train_acc)
            print("Validation accuracy: %.1f%%" % valid_acc)
    
    plt.plot(step_hist, train_accs, 'b', label='training')
    plt.plot(step_hist, valid_accs, 'g', label='validation')
    plt.legend(loc="lower right")
    plt.show()
    print("Test accuracy: %.1f%%" % accuracy(
            test_prediction.eval(), test_labels))

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [ ]:
# Define the graph
###
batch_size = 128
fc1_nodes = 1024
lr = 0.03
keep_prob = 0.8
lam = 0.001

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, 
                                      [batch_size, image_size * image_size])
    tf_train_labels = tf.placeholder(tf.float32, 
                                     [batch_size, num_labels])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    fc1_weight = tf.Variable(tf.truncated_normal([image_size * image_size, fc1_nodes]))
    fc1_bias = tf.Variable(tf.zeros([fc1_nodes]))
    
    fc2_weight = tf.Variable(tf.truncated_normal([fc1_nodes, num_labels]))
    fc2_bias = tf.Variable(tf.zeros([num_labels]))
    
    def model(data, train=False):
        h1 = tf.nn.relu(tf.matmul(data, fc1_weight) + fc1_bias)
        if train:
            h1 = tf.nn.dropout(h1, keep_prob)
            
        logits = tf.matmul(h1, fc2_weight) + fc2_bias
        return logits
    
    logits = model(tf_train_dataset, True)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    regularizer = (tf.nn.l2_loss(fc1_weight) + tf.nn.l2_loss(fc1_bias) 
                   + tf.nn.l2_loss(fc2_weight) + tf.nn.l2_loss(fc2_bias))
    loss += lam * regularizer
    optimizer = tf.train.GradientDescentOptimizer(lr).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

###
# Run the graph
###
train_size = train_dataset.shape[0]
num_steps = train_size // batch_size * 100

with tf.Session(graph=graph) as sess:
    tf.global_variables_initializer().run()
    train_accs, valid_accs, step_hist = [], [], []
    for step in range(num_steps):
        offset = (step * batch_size) % (train_size - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, 
                     tf_train_labels: batch_labels}
        _, l, predictions = sess.run([optimizer, loss, train_prediction],
                                     feed_dict=feed_dict)
        if (step % 500 == 0):
            train_acc = accuracy(predictions, batch_labels)
            valid_acc = accuracy(valid_prediction.eval(), valid_labels)
            train_accs.append(train_acc / 100.0)
            valid_accs.append(valid_acc / 100.0)
            step_hist.append(step)
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % train_acc)
            print("Validation accuracy: %.1f%%" % valid_acc)
    
    plt.plot(step_hist, train_accs, 'b', label='training')
    plt.plot(step_hist, valid_accs, 'g', label='validation')
    plt.legend(loc="lower right")
    plt.show()
    print("Test accuracy: %.1f%%" % accuracy(
            test_prediction.eval(), test_labels))

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [ ]:
###
# Define the graph
###

# In current setting, it's ok to increase layer size and increase dropout.
batch_size = 1024
fc1_nodes = 1024
fc2_nodes = 512
fc3_nodes = 256
keep_prob = 0.8
lr = 0.001
lam = 0.001

graph = tf.Graph()
with graph.as_default():    
    tf_train_dataset = tf.placeholder(tf.float32, 
                                      [batch_size, image_size * image_size])
    tf_train_labels = tf.placeholder(tf.float32, 
                                     [batch_size, num_labels])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    '''
    fc1_weight = tf.Variable(tf.truncated_normal([image_size * image_size, fc1_nodes]))
    fc1_bias = tf.Variable(tf.zeros([fc1_nodes]))

    fc2_weight = tf.Variable(tf.truncated_normal([fc1_nodes, fc2_nodes]))
    fc2_bias = tf.Variable(tf.zeros([fc2_nodes]))

    fc3_weight = tf.Variable(tf.truncated_normal([fc2_nodes, fc3_nodes]))
    fc3_bias = tf.Variable(tf.zeros([fc3_nodes]))

    fc4_weight = tf.Variable(tf.truncated_normal([fc3_nodes, num_labels]))
    fc4_bias = tf.Variable(tf.zeros([num_labels]))
    '''
    fc1_weight = tf.get_variable('fc1_weight', shape=[image_size * image_size, fc1_nodes], 
                                initializer=tf.contrib.layers.xavier_initializer())
    fc1_bias = tf.get_variable('fc1_bias', initializer=tf.zeros([fc1_nodes]))
    
    fc2_weight = tf.get_variable('fc2_weight', shape=[fc1_nodes, fc2_nodes], 
                                initializer=tf.contrib.layers.xavier_initializer())
    fc2_bias = tf.get_variable('fc2_bias', initializer=tf.zeros([fc2_nodes]))
    
    fc3_weight = tf.get_variable('fc3_weight', shape=[fc2_nodes, fc3_nodes],
                                initializer=tf.contrib.layers.xavier_initializer())
    fc3_bias = tf.get_variable('fc3_bias', initializer=tf.zeros([fc3_nodes]))
    
    fc4_weight = tf.get_variable('fc4_weight', shape=[fc3_nodes, num_labels], 
                                initializer=tf.contrib.layers.xavier_initializer())
    fc4_bias = tf.get_variable('fc4_bias', initializer=tf.zeros([num_labels]))
    

    def model(data, train=False):
        h1 = tf.nn.relu(tf.matmul(data, fc1_weight) + fc1_bias)
        if train:
            h1 = tf.nn.dropout(h1, keep_prob)

        h2 = tf.nn.relu(tf.matmul(h1, fc2_weight) + fc2_bias)
        if train:
            h2 = tf.nn.dropout(h2, keep_prob)

        h3 = tf.nn.relu(tf.matmul(h2, fc3_weight) + fc3_bias)
        if train:
            h3 = tf.nn.dropout(h3, keep_prob)

        logits = tf.matmul(h3, fc4_weight) + fc4_bias
        return logits

    logits = model(tf_train_dataset, True)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    '''
    regularizer = (tf.nn.l2_loss(fc1_weight) + tf.nn.l2_loss(fc1_bias) 
                   + tf.nn.l2_loss(fc2_weight) + tf.nn.l2_loss(fc2_bias))
    loss += lam * regularizer
    '''
    optimizer = tf.train.AdagradOptimizer(lr).minimize(loss)

    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

###
# Run the graph
###
train_size = train_dataset.shape[0]
num_steps = train_size // batch_size * 1000

with tf.Session(graph=graph) as sess:
    tf.global_variables_initializer().run()
    train_accs, valid_accs, step_hist = [], [], []
    for step in range(num_steps):
        offset = (step * batch_size) % (train_size - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, 
                     tf_train_labels: batch_labels}
        _, l, predictions, the_logits = sess.run([optimizer, loss, train_prediction, logits],
                                     feed_dict=feed_dict)
        if (step % 500 == 0):
            train_acc = accuracy(predictions, batch_labels)
            valid_acc = accuracy(valid_prediction.eval(), valid_labels)
            train_accs.append(train_acc / 100.0)
            valid_accs.append(valid_acc / 100.0)
            step_hist.append(step)
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % train_acc)
            print("Validation accuracy: %.1f%%" % valid_acc)
    
    plt.plot(step_hist, train_accs, 'b', label='training')
    plt.plot(step_hist, valid_accs, 'g', label='validation')
    plt.legend(loc="lower right")
    plt.show()
    print("Test accuracy: %.1f%%" % accuracy(
            test_prediction.eval(), test_labels))